In [1]:
%pip uninstall numpy -y
%pip install --upgrade numpy

Found existing installation: numpy 2.2.5
Uninstalling numpy-2.2.5:
  Successfully uninstalled numpy-2.2.5
Note: you may need to restart the kernel to use updated packages.
  Using cached numpy-2.2.5-cp312-cp312-macosx_14_0_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp312-cp312-macosx_14_0_x86_64.whl (6.7 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.5 which is incompatible.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
gpy 1.13.2 requires numpy<2.0.0,>=1.7, but you have numpy 2.2.5 which is incompatible.
scipy 1.12.0 requires numpy<1.29.0,>=1.22.4, but you have numpy 2.2.5 which is incompatible.
Note: you may need to restart the kernel to use updated 

In [3]:
%pip install "numpy<2" --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 9.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
Note: you may need to restart the kernel to use updated packages.


In [ ]:
'''
This python script performs 10-fold cross validation for a dataset for 7 ML methods and outputs:
1. The resulting average metrics
2. Predicted vs experimental values for all folds and models
'''
#section 1: import modules
import sys,os,re
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.cross_decomposition import PLSRegression
from sklearn import ensemble
from scipy.stats import pearsonr
import math
from sklearn.model_selection import KFold
import statistics

##section 2: define inputs and outputs
dir = os.getcwd() 
Dataset=pd.read_csv(os.path.join(dir,"/Users/stella/Downloads/tierney/project/all_final_data.csv")) #location of descriptor file
output_metrics=os.path.join(dir,"/Users/stella/Downloads/tierney/project/MLCV_metrics.csv") #location of output file for metrics
output_predictions=os.path.join(dir,"/Users/stella/Downloads/tierney/project/MLCV_LogS_predictions.csv") #new file for predictions

##section 3: define methods
#Define statistical measures and R2 conversion
#define RMSE
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
#define % within certain range
def within_range(list1, list2, range2):
    x=0
    for i in range(len(list2)):
        if (list1[i]-range2)<= list2[i] <= (list1[i]+range2): 
            x+=1
    return((float(x)/(len(list2)))*100)
#define getting R2 method
def get_R2(R2):
    R2_2=[]
    for i in range(len(R2)):
        x=re.findall('\d\.\d+',str(R2[i]))
        j=float(x[0])
        j=j**2
        R2_2.append(j)
    return(R2_2)
#define method to get CV results
def CV_metrics(Data,folds,C,E,G):
    #initiate lists to add metrics to
    RMSE=[]
    R2=[]
    N1=[]
    N05=[]
    MLR_RMSE=[]
    MLR_R2=[]
    MLR_N1=[]
    MLR_N05=[]
    ANN_RMSE=[]
    ANN_R2=[]
    ANN_N1=[]
    ANN_N05=[]
    SVM_RMSE=[]
    SVM_R2=[]
    SVM_N1=[]
    SVM_N05=[]
    PLS_RMSE=[]
    PLS_R2=[]
    PLS_N1=[]
    PLS_N05=[]
    RF_RMSE=[]
    RF_R2=[]
    RF_N1=[]
    RF_N05=[]
    ET_RMSE=[]
    ET_R2=[]
    ET_N1=[]
    ET_N05=[]
    BG_RMSE=[]
    BG_R2=[]
    BG_N1=[]
    BG_N05=[]
    
    # Initialize lists to store predictions and experimental values
    all_preds = []
    
    #import Data and randomise
    X = Data
    X = X.sample(frac=1).reset_index(drop=True)
    #define k-fold cross validation and make k splits
    col_names=X.dtypes.index
    X = np.array(X)
    kf = KFold(n_splits=folds)
    fold_num = 1
    
    #for every split
    for train1, test1 in kf.split(X):
        train=X[train1]
        test=X[test1]
        train=pd.DataFrame(data=train, columns=col_names)
        test=pd.DataFrame(data=test, columns=col_names)
        
        # Get compound identifiers and solvent information
        inchikeys = test['InChIkey'] if 'InChIkey' in test.columns else ['Unknown']*len(test)
        solvents = test['Solvent'] if 'Solvent' in test.columns else ['Unknown']*len(test)
        
        X_train = train[['MW', 'Volume', 'G_sol', 'DeltaG_sol', 'sol_dip',
                         'LsoluHsolv', 'LsolvHsolu', 'SASA', 'O_charges',
                         'C_charges', 'Most_neg', 'Most_pos', 'Het_charges']]
        y_train = train['LogS']
        X_test = test[['MW', 'Volume', 'G_sol', 'DeltaG_sol', 'sol_dip',
                       'LsoluHsolv', 'LsolvHsolu', 'SASA', 'O_charges',
                       'C_charges', 'Most_neg', 'Most_pos', 'Het_charges']]
        y_test = test['LogS']
        
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
    
        # Create dictionary to store this fold's predictions
        fold_preds = {
            'Fold': [fold_num] * len(y_test),
            'InChIkey': inchikeys,
            'Solvent': solvents,
            'Experimental': y_test,
        }
        
        #run models
        #MLR
        mlr = LinearRegression()
        mlr.fit(X_train, y_train)
        mlr2preds = mlr.predict(X_test)
        fold_preds['MLR'] = mlr2preds
        #evaluate model
        MLR_R2.append(pearsonr(mlr2preds, y_test))
        MLR_RMSE.append(rmse(mlr2preds, y_test))
        MLR_N1.append(within_range(y_test,mlr2preds,1))
        MLR_N05.append(within_range(y_test,mlr2preds,0.7))
        
        #ANN
        mlp = MLPRegressor(hidden_layer_sizes=300,max_iter=800)
        for f in range(100):
            try:
                mlp.fit(X_train, y_train)
                mlp2preds = mlp.predict(X_test)
                if np.ptp(mlp2preds) == 0:
                    continue
                break
            except:
                continue
        fold_preds['ANN'] = mlp2preds
        #evaluate model
        ANN_R2.append(pearsonr(mlp2preds, y_test))
        ANN_RMSE.append(rmse(mlp2preds, y_test))
        ANN_N1.append(within_range(y_test,mlp2preds,1))
        ANN_N05.append(within_range(y_test,mlp2preds,0.7))
        
        #SVM
        svm2 = svm.SVR(C = C, epsilon = E, gamma = G, kernel = 'rbf')
        svm2.fit(X_train, y_train)
        svm2preds = svm2.predict(X_test)
        fold_preds['SVM'] = svm2preds
        #evaluate model
        SVM_R2.append(pearsonr(svm2preds, y_test))
        SVM_RMSE.append(rmse(svm2preds, y_test))
        SVM_N1.append(within_range(y_test,svm2preds,1))
        SVM_N05.append(within_range(y_test,svm2preds,0.7))
        
        #PLS
        pls2 = PLSRegression(n_components=9)
        pls2.fit(X_train, y_train)
        pls2preds = pls2.predict(X_test)
        #convert to float (comes in weird type?)
        pls2preds2=[]
        for i in pls2preds:
            pls2preds2.append(float(i))
        fold_preds['PLS'] = pls2preds2
        #evaluate model
        PLS_R2.append(pearsonr(pls2preds2, y_test))
        PLS_RMSE.append(rmse(pls2preds2, y_test))
        PLS_N1.append(within_range(y_test,pls2preds2,1))
        PLS_N05.append(within_range(y_test,pls2preds2,0.7))
        
        #RF
        tree2 = ensemble.RandomForestRegressor(n_estimators=500, n_jobs=1)
        tree2.fit(X_train, y_train)
        tree2preds = tree2.predict(X_test)
        fold_preds['RF'] = tree2preds
        #evaluate model
        RF_R2.append(pearsonr(tree2preds, y_test))
        RF_RMSE.append(rmse(tree2preds, y_test))
        RF_N1.append(within_range(y_test, tree2preds, 1))
        RF_N05.append(within_range(y_test, tree2preds, 0.7))
        
        #ExtraTrees
        tree3 = ensemble.ExtraTreesRegressor(n_estimators=500, n_jobs=1)
        tree3.fit(X_train, y_train)
        tree3preds = tree3.predict(X_test)
        fold_preds['ExtraTrees'] = tree3preds
        #evaluate model
        ET_R2.append(pearsonr(tree3preds, y_test))
        ET_RMSE.append(rmse(tree3preds, y_test))
        ET_N1.append(within_range(y_test, tree3preds, 1))
        ET_N05.append(within_range(y_test, tree3preds, 0.7))
        
        #Bagging
        tree4 = ensemble.BaggingRegressor(n_estimators=500, n_jobs=1)
        tree4.fit(X_train, y_train)
        tree4preds = tree4.predict(X_test)
        fold_preds['Bagging'] = tree4preds
        #evaluate model
        BG_R2.append(pearsonr(tree4preds, y_test))
        BG_RMSE.append(rmse(tree4preds, y_test))
        BG_N1.append(within_range(y_test, tree4preds, 1))
        BG_N05.append(within_range(y_test, tree4preds, 0.7))
        
        # Add this fold's predictions to the main list
        all_preds.append(pd.DataFrame(fold_preds))
        fold_num += 1
    
    # Combine all fold predictions into one DataFrame
    all_preds_df = pd.concat(all_preds, ignore_index=True)
    
    #get R2 from Pearson output
    MLR_R2=get_R2(MLR_R2)
    ANN_R2=get_R2(ANN_R2)
    SVM_R2=get_R2(SVM_R2)
    PLS_R2=get_R2(PLS_R2)
    RF_R2=get_R2(RF_R2)
    ET_R2=get_R2(ET_R2)
    BG_R2=get_R2(BG_R2)
    
    #get mean metrics and put together in lists
    R2.append(statistics.mean(MLR_R2))
    RMSE.append(statistics.mean(MLR_RMSE))
    N1.append(statistics.mean(MLR_N1))
    N05.append(statistics.mean(MLR_N05))
    
    R2.append(statistics.mean(ANN_R2))
    RMSE.append(statistics.mean(ANN_RMSE))
    N1.append(statistics.mean(ANN_N1))
    N05.append(statistics.mean(ANN_N05))
    
    R2.append(statistics.mean(SVM_R2))
    RMSE.append(statistics.mean(SVM_RMSE))
    N1.append(statistics.mean(SVM_N1))
    N05.append(statistics.mean(SVM_N05))
    
    R2.append(statistics.mean(PLS_R2))
    RMSE.append(statistics.mean(PLS_RMSE))
    N1.append(statistics.mean(PLS_N1))
    N05.append(statistics.mean(PLS_N05))
    
    R2.append(statistics.mean(RF_R2))
    RMSE.append(statistics.mean(RF_RMSE))
    N1.append(statistics.mean(RF_N1))
    N05.append(statistics.mean(RF_N05))
    
    R2.append(statistics.mean(ET_R2))
    RMSE.append(statistics.mean(ET_RMSE))
    N1.append(statistics.mean(ET_N1))
    N05.append(statistics.mean(ET_N05))
    
    R2.append(statistics.mean(BG_R2))
    RMSE.append(statistics.mean(BG_RMSE))
    N1.append(statistics.mean(BG_N1))
    N05.append(statistics.mean(BG_N05))
    
    #create dataframe of metrics
    Models=["MLR","ANN","SVM","PLS","RF","ExtraTrees","Bagging"]
    Metrics=list(zip(Models,R2,RMSE,N1,N05))
    Metrics_df=pd.DataFrame(data=Metrics, columns=['Model','R2','RMSE','% within 1','% within 0.7'])
    
    return Metrics_df, all_preds_df

##method to put it all together
def get_CV_metrics(Dataset,output_metrics,output_predictions):
    ##get metrics and predictions
    CV_metrics2, all_predictions = CV_metrics(Dataset,10,4,0.01,0.03) ##10-folds and SVM parameters
    
    ##save to files
    CV_metrics2.to_csv(output_metrics,index=False)
    all_predictions.to_csv(output_predictions,index=False)

##section 4: run CV method and get metrics and predictions
get_CV_metrics(Dataset,output_metrics,output_predictions)

<>:43: SyntaxWarning: invalid escape sequence '\d'
<>:43: SyntaxWarning: invalid escape sequence '\d'
/var/folders/m2/ptj22ckn1lq9dwc6k6bwvz580000gn/T/ipykernel_43069/1911255660.py:43: SyntaxWarning: invalid escape sequence '\d'
  x=re.findall('\d\.\d+',str(R2[i]))


In [ ]:
import pandas as pd

# File paths
gpr_file = "/Users/stella/Downloads/tierney/project/GPCV_predictions.csv"
ml_file = "/Users/stella/Downloads/tierney/project/MLCV_LogS_predictions.csv"
output_file = "/Users/stella/Downloads/tierney/project/all_predictions.csv"

# Read files with clean column names
gpr_preds = pd.read_csv(gpr_file)
ml_preds = pd.read_csv(ml_file)

# Clean column names
gpr_preds.columns = gpr_preds.columns.str.strip()
ml_preds.columns = ml_preds.columns.str.strip()

# Verify required columns exist
required = ['InChIkey', 'Solvent']
for col in required:
    if col not in gpr_preds.columns:
        raise ValueError(f"'{col}' missing in GPR file")
    if col not in ml_preds.columns:
        raise ValueError(f"'{col}' missing in ML file")

# Perform merge

combined = pd.merge(
    ml_preds,
    gpr_preds[['InChIkey','Solvent','Predicted']],
    on=['InChIkey','Solvent'],  # All 3 columns
    how='left'
)

# Check results
print(f"Merged {len(ml_preds)} ML rows with GPR predictions")
print(f"Final dataset has {len(combined)} rows")
missing = combined['Predicted'].isna().sum()
if missing > 0:
    print(f"Warning: {missing} rows missing GPR predictions")

# Save output
combined.to_csv(output_file, index=False)
print(f"\nSaved to {output_file}")
print("Final columns:", combined.columns.tolist())

Merged 3249 ML rows with GPR predictions
Final dataset has 3249 rows

Saved to /Users/stella/Downloads/tierney/project/all_predictions.csv
Final columns: ['Fold', 'InChIkey', 'Solvent', 'Experimental', 'MLR', 'ANN', 'SVM', 'PLS', 'RF', 'ExtraTrees', 'Bagging', 'Predicted']
